In [1]:
import pickle 
import os
import torch
import os.path


In [3]:
# %%writefile merge.py
# import argparse
# import os

# parser = argparse.ArgumentParser("dot")
# parser.add_argument("dataset_folder", help="Path to a dataset folder of .train files that can be read by calling load_dataset('text', <path>)")
# parser.add_argument("model_dir", help="Where the model and checkpoints are stored")
# parser.add_argument("gradient_input_dir", help="Where the gradients are stored")
# parser.add_argument("influence_output_dir", help="Where the influence scores should be stored")

# parser.add_argument("--num_processes", help="Number of processes to use (one model per process)", type=int, nargs="?", const=1, default=10)
# parser.add_argument("--cuda_visible_devices", help="Comma seperated GPU ids to use", nargs="?", const=1, default="0,1")
# parser.add_argument("--gradients_per_file", help="Number of gradients per output file", type=int, nargs="?", const=1, default=10000)
# args = parser.parse_args()

# if not os.path.exists(args.influence_output_dir):
#     os.makedirs(args.influence_output_dir)

# os.environ["CUDA_VISIBLE_DEVICES"] = args.cuda_visible_devices
# os.environ["TOKENIZERS_PARALLELISM"] = "False"

# import traceback

# import torch 
# import time
# def run(chunk_path_a, start_id_a, stop_id_a, subtasks):

#     # from tqdm import tqdm

#     device = "cpu"
    
#     try:
#         with torch.no_grad():
#             start_time = time.time()
#             chunk_a = torch.load(chunk_path_a, weights_only=True,map_location=device).flatten(1)
#             print(f"Time to load chunk_a: {time.time() - start_time:.4f} seconds", flush=True)
#             result = torch.zeros((chunk_a.shape[0])).to(device)
#             for chunk_path_b,start_id_b, stop_id_b in  tqdm(subtasks, leave=False):
#                 chunk_b = torch.load(chunk_path_b, weights_only=True,map_location=device).flatten(1)
#                 start_time = time.time()
#                 result  += torch.einsum('ik, kj -> i', chunk_a, chunk_b.T)
#                 print(f"Time to einsum: {time.time() - start_time:.4f} seconds", flush=True)
#             return (start_id_a, stop_id_a, result)

#     except Exception as e:
#         print(traceback.format_exc(),flush=True)
#         raise e

# from multiprocessing import Pool, current_process
# import time 
# import datetime
# import os
# from pathlib import Path
# import torch
# from itertools import cycle



# from multiprocessing import Pool,Queue,Manager
# import multiprocessing as mp

# if __name__ == '__main__':
#     # mp.set_start_method('fork')

#     from transformers import RobertaConfig,AutoConfig
#     from transformers import RobertaForMaskedLM
#     import torch
#     from tqdm import tqdm
#     from transformers import RobertaTokenizerFast
#     from transformers import DataCollatorForLanguageModeling
#     tokenizer = RobertaTokenizerFast.from_pretrained(args.model_dir, max_len=512)
#     data_collator = DataCollatorForLanguageModeling(
#         tokenizer=tokenizer, mlm=True, mlm_probability=0.15
#     )
#     from datasets import load_dataset
#     dataset = load_dataset("text", data_dir=args.dataset_folder)
#     dataset.set_transform(lambda x : tokenizer(x["text"], return_special_tokens_mask=True, truncation=True, padding="max_length", max_length=512))


    
#     def get_all_chunks(checkpoint_path):
#         # return [ os.path.join(args.gradient_input_dir, checkpoint_path.split("-")[-1] + "_" + str(i) + "_" + str(i + args.gradients_per_file)) for i in range(0, len(dataset["train"]), args.gradients_per_file)]
#         return [str(x) for x in Path(args.gradient_input_dir).glob("*")]






#     with Pool(args.num_processes) as p:
#         from util import get_epoch_checkpoints
#         checkpoints = [str(x) for x in Path(args.model_dir).glob("checkpoint-*") if int(str(x).split("-")[-1]) in get_epoch_checkpoints(args.model_dir)]
        
#         for checkpoint_path in tqdm(checkpoints, desc="Running for checkpoints", leave=False):
#             result_checkpoint = torch.zeros((len(dataset["train"])))
#             out_path = os.path.join(args.influence_output_dir, checkpoint_path.split("-")[-1])
#             if os.path.isfile(out_path):
#                 True
#                 continue

#             tasks = []
#             for chunk_path_a in get_all_chunks(checkpoint_path):
#                 _, start_id_a, stop_id_a = chunk_path_a.split( "_")
#                 start_id_a = int(start_id_a)
#                 stop_id_a = int(stop_id_a)
#                 subtasks = []
#                 for chunk_path_b in get_all_chunks(checkpoint_path):
#                     _, start_id_b, stop_id_b = chunk_path_b.split( "_")
#                     start_id_b = int(start_id_b)
#                     stop_id_b = int(stop_id_b)
#                     subtasks.append((chunk_path_b,start_id_b, stop_id_b))
#                 tasks.append((chunk_path_a, start_id_a, stop_id_a, subtasks))
#             tasks = tasks[0:1000]
#             r = p.starmap(run, tasks )
#             for start_id_a, _, rr in r:
#                 result_checkpoint[start_id_a:start_id_a + rr.shape[0]] += rr #  the stop_ids are taken from the task description in if.ipynb and can therefore be higher than the actual lenght
#             result_checkpoint = (result_checkpoint / len(dataset["train"])).unsqueeze(0)   
#             torch.save(result_checkpoint, out_path)


Overwriting merge.py


In [73]:
%%writefile merge.py
import argparse
import os

parser = argparse.ArgumentParser("dot")
parser.add_argument("dataset_folder", help="Path to a dataset folder of .train files that can be read by calling load_dataset('text', <path>)")
parser.add_argument("model_dir", help="Where the model and checkpoints are stored")
parser.add_argument("gradient_input_dir", help="Where the gradients are stored")
parser.add_argument("influence_output_dir", help="Where the influence scores should be stored")

parser.add_argument("--num_processes", help="Number of processes to use (one model per process)", type=int, nargs="?", const=1, default=4)
parser.add_argument("--cuda_visible_devices", help="Comma seperated GPU ids to use", nargs="?", const=1, default="0,1")
parser.add_argument("--gradients_per_file", help="Number of gradients per output file", type=int, nargs="?", const=1, default=10000)
parser.add_argument("--batch_size", help="How many chunks each subprocess will keep in memory", type=int, nargs="?", const=1, default=20)
args = parser.parse_args()

if not os.path.exists(args.influence_output_dir):
    os.makedirs(args.influence_output_dir)

os.environ["CUDA_VISIBLE_DEVICES"] = args.cuda_visible_devices
os.environ["TOKENIZERS_PARALLELISM"] = "False"

import traceback

import logging
logging.basicConfig(filename="merge.log",
                    level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

import torch 
import time
def run(tasks, subtasks):

    # from tqdm import tqdm

    device = "cpu"
    
    try:
        with torch.no_grad():
            start_time = time.time()
            try:
                chunks_a = [torch.load(chunk_path_a, weights_only=True,map_location=device).flatten(1) for (chunk_path_a,start_id_a, stop_id_a) in tasks]
            except:
                logging.error("{} seems corrupted (recompute!)".format(tasks) ) # this happens if the gradient extraction script was killed during torch.save
                return None
            logging.info(f"Time to load task: {time.time() - start_time:.4f} seconds")
            results = [torch.zeros((chunk_a.shape[0])).to(device) for chunk_a in chunks_a]
            start_time = time.time()
            for chunk_path_b,start_id_b, stop_id_b in subtasks:
                try:
                    chunk_b = torch.load(chunk_path_b, weights_only=True,map_location=device).flatten(1)
                except:
                    # this happens if the gradient extraction script was killed during torch.save
                    logging.error("{} seems corrupted (recompute!)".format(chunk_path_b) )
                    return None
                for i, chunk_a in enumerate(chunks_a):
                    results[i]  += torch.einsum('ik, kj -> i', chunk_a, chunk_b.T)
            logging.info(f"Time to einsum: {time.time() - start_time:.4f} seconds; {(time.time() - start_time)/len(subtasks):.4f} s/chunk")

            return (tasks, results)

    except Exception as e:
        print(traceback.format_exc(),flush=True)
        raise e

from multiprocessing import Pool, current_process
import time 
import datetime
import os
from pathlib import Path
import torch
from itertools import cycle



from multiprocessing import Pool,Queue,Manager
import multiprocessing as mp

if __name__ == '__main__':
    # mp.set_start_method('fork')

    from transformers import RobertaConfig,AutoConfig
    from transformers import RobertaForMaskedLM
    import torch
    from tqdm import tqdm
    from transformers import RobertaTokenizerFast
    from transformers import DataCollatorForLanguageModeling
    tokenizer = RobertaTokenizerFast.from_pretrained(args.model_dir, max_len=512)
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True, mlm_probability=0.15
    )
    from datasets import load_dataset
    dataset = load_dataset("text", data_dir=args.dataset_folder)
    dataset.set_transform(lambda x : tokenizer(x["text"], return_special_tokens_mask=True, truncation=True, padding="max_length", max_length=512))

    def batch(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]
        
    def get_all_chunks(checkpoint_path):
        return [ os.path.join(args.gradient_input_dir, checkpoint_path.split("-")[-1] + "_" + str(i) + "_" + str(i + args.gradients_per_file)) for i in range(0, len(dataset["train"]), args.gradients_per_file)]
        
        #return [str(x) for x in Path(checkpoint_path).glob("*")]






    with Pool(args.num_processes) as p:
        from util import get_epoch_checkpoints
        checkpoints = [str(x) for x in Path(args.model_dir).glob("checkpoint-*") if int(str(x).split("-")[-1]) in get_epoch_checkpoints(args.model_dir)]
        
        for checkpoint_path in tqdm(checkpoints, desc="Running for checkpoints", leave=False):
            result_checkpoint = torch.zeros((len(dataset["train"])))
            out_path = os.path.join(args.influence_output_dir, checkpoint_path.split("-")[-1])
            if os.path.isfile(out_path):
                continue

            jobs = []
            
            # tasks = []
            # for chunk_path_a in get_all_chunks(checkpoint_path):
            #     _, start_id_a, stop_id_a = chunk_path_a.split( "_")
            #     start_id_a = int(start_id_a)
            #     stop_id_a = int(stop_id_a)
            #     tasks.append((chunk_path_a, start_id_a, stop_id_a))

            print("that many chunks in this checkpoint:", len(get_all_chunks(checkpoint_path)), flush=True)
            subtasks = []
            for chunk_path_b in get_all_chunks(checkpoint_path):
                _, start_id_b, stop_id_b = chunk_path_b.split( "_")
                start_id_b = int(start_id_b)
                stop_id_b = int(stop_id_b)
                subtasks.append((chunk_path_b,start_id_b, stop_id_b))
           # subtasks = subtasks[0:100]
            for tasks in batch(subtasks, args.batch_size):
                jobs.append((tasks, subtasks))
            #jobs = jobs[0:1]
            
            # exit
            r = p.starmap(run, jobs )
            for rr in r:
                for task, result in zip(*rr):
                    chunk_path_a, start_id_a, stop_id_a = task
                    result_checkpoint[start_id_a:(start_id_a + result.shape[0])] += result #  the stop_ids are taken from the task description in if.ipynb and can therefore be higher than the actual lenght
            result_checkpoint = (result_checkpoint / len(dataset["train"])).unsqueeze(0)   
            torch.save(result_checkpoint, out_path)
            logging.info("Saved {}".format(out_path))

Overwriting merge.py


In [37]:
import numpy as np
a = np.random.rand(10000, 393216)
b = np.random.rand(393216,10000)
r = np.einsum_path('ik, kj -> i', a, b, optimize='optimal')

In [34]:
r[0]

['einsum_path', (0, 1)]

In [38]:
print(r[1])

  Complete contraction:  ik,kj->i
         Naive scaling:  3
     Optimized scaling:  3
      Naive FLOP count:  7.864e+13
  Optimized FLOP count:  7.864e+13
   Theoretical speedup:  1.000
  Largest intermediate:  1.000e+04 elements
--------------------------------------------------------------------------
scaling                  current                                remaining
--------------------------------------------------------------------------
   3                    kj,ik->i                                     i->i


In [3]:
# %run merge.py ./train_10M ./10MModel ./gradients ./influence

In [5]:
### Time Estimate
For 10M 10 checkpoints (one per epoch)

SyntaxError: invalid decimal literal (3516313529.py, line 2)

## Create New Curriculum

In [4]:
args = {
    "dataset_folder": "./train_10M",
    "model_dir": "./10MModel",
    "influence_output_dir": "./influence",
    "curriculum_output_folder": "./10MCurriculum"
}

In [5]:
from pathlib import Path

from datasets import load_dataset
dataset = load_dataset("text", data_dir=args["dataset_folder"])
#dataset.set_transform(lambda x : tokenizer(x["text"], return_special_tokens_mask=True, truncation=True, padding="max_length", max_length=512))

((len(dataset["train"]) / 10000) / 10)*7

82.53098

In [6]:
import pandas as  pd
import os
import torch
import numpy as np
# pd.DataFrame(a)

In [7]:
if not os.path.exists(args["curriculum_output_folder"]):
    os.makedirs(args["curriculum_output_folder"])

In [8]:

df = pd.DataFrame({int(result_checkpoint): torch.load(os.path.join(args["influence_output_dir"],result_checkpoint),weights_only=True,map_location="cpu").numpy().flatten() for result_checkpoint in os.listdir(args["influence_output_dir"])})
# df = df.rename({0: "influence"}, axis=1)
df.sort_index(axis=1)

STAGES = list(df.columns)
NUM_STAGES = len(STAGES)

df = df.reindex(sorted(df.columns, reverse=False), axis=1)
df["total"] = df.sum(axis=1)
df["text"] = dataset["train"].to_pandas()
df

,375,750,1125,1500,1875,2250,total,text
0,4.849778,4.792889,5.091556,5.176889,4.622222,4.650667,29.184002,A:\tI'm sure they are.
1,0.079444,0.012611,0.097889,0.068111,0.038500,0.050056,0.346611,"A:\tThat's right,"
2,1.045333,1.098667,0.888889,1.041778,1.009778,0.890667,5.975111,A:\tI'm sure that true.
3,2.225778,2.161778,2.133333,2.168889,1.984000,2.296889,12.970667,A:\tIt's a lot different than working in a kid...
4,0.194667,0.165778,0.232000,0.326222,0.284889,0.291556,1.495111,B:\tA lot different.
...,...,...,...,...,...,...,...,...
17995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,B:\tThat's right.
17996,0.383111,0.376000,0.376889,0.381333,0.386667,0.396444,2.300445,"A:\tAnd, and the schools don't, don't really e..."
17997,-0.008375,0.003253,-0.006597,-0.012722,-0.012222,0.006097,-0.030566,A:\tthe schools are there to teach history and...
17998,5.134222,5.304889,4.892445,5.361778,4.849778,4.821333,30.364443,A:\tThey don't teach them good values like dru...


In [9]:
df[17990:]

,375,750,1125,1500,1875,2250,total,text
17990,0.011347,0.027056,0.009431,0.001181,0.015056,0.012583,0.076653,"A:\tYou got to, you got to be your child's bes..."
17991,1.795556,1.952000,1.976889,1.834667,1.916444,1.724444,11.200000,B:\tRight.
17992,0.047333,0.060889,0.056556,0.066000,0.055889,0.054889,0.341556,A:\tstill you got to be their best friend beca...
17993,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,B:\tExactly.
17994,0.029333,0.050444,0.041000,0.041111,0.034889,0.031333,0.228111,"A:\tan, and if they go to their friends dealin..."
17995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,B:\tThat's right.
17996,0.383111,0.376000,0.376889,0.381333,0.386667,0.396444,2.300445,"A:\tAnd, and the schools don't, don't really e..."
17997,-0.008375,0.003253,-0.006597,-0.012722,-0.012222,0.006097,-0.030566,A:\tthe schools are there to teach history and...
17998,5.134222,5.304889,4.892445,5.361778,4.849778,4.821333,30.364443,A:\tThey don't teach them good values like dru...
17999,0.041333,0.055000,0.043667,0.052778,0.078444,0.048778,0.320000,"A:\tI, maybe I'm wrong, because I haven't, you..."


In [10]:
NUM_DOCS_STAGE = len(df) // NUM_STAGES
NUM_DOCS_STAGE

3000

In [11]:
from scipy.stats import norm
from scipy.stats import f
from scipy.stats import lognorm

: 

In [12]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
size=len(STAGES)
print(size)
def gaussian_filter(size, **args):
    print(args)
    indices = np.arange(-size, size+1, 1)
    weights =  norm.pdf(indices, **args)
    return weights# / np.sum(weights)


def f_filter(size, **args):
    print(args)
    indices = np.arange(-size, size+1, 1)
    print(indices)
    weights = f.pdf(indices+1, **args)
    return weights# / np.sum(weights)
def lognorm_filter(size, **args):
    #print(args)
    indices = np.arange(-size, size+1, 1)
    #print(indices)
    weights = lognorm.pdf(indices+1, **args)
    return weights / np.sum(weights)




: 

In [13]:
# filter_weights = f_filter(size, dfn=15, dfd=3)
# print(filter_weights)
# plt.plot(np.arange(-size,size, 1),filter_weights)
# plt.vlines(0, ymin=0, ymax=max(filter_weights)+0.1, colors=["red"])

: 

In [14]:

filter_weights = lognorm_filter(size, s=1, loc=0, scale=0.5)
plt.plot(np.arange(-size,size+1, 1),filter_weights)
plt.vlines([0], ymin=0, ymax=max(filter_weights)+0.1, colors=["red"])


: 

In [15]:
filter_weights

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.71524275, 0.17395476, 0.06088795, 0.02616743,
       0.01283867, 0.00691511, 0.00399333])

In [16]:
NUM_STAGES

6

In [17]:
a = [1000,10,1,]
np.convolve(a,filter_weights, mode="full")

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 7.15242749e+02, 1.81107192e+02,
       6.33427358e+01, 2.69502660e+01, 1.31612300e+01, 7.06966155e+00,
       4.07532409e+00, 4.68484509e-02, 3.99333434e-03])

In [18]:
n = df[STAGES].to_numpy()#[0:1000,:]
scores = np.apply_along_axis(lambda m: np.convolve(m,filter_weights, mode="valid")[1:-1], axis=1, arr=n)
scores.shape

(18000, 6)

In [19]:
df_usefulness = df.copy()
df_usefulness[STAGES] = np.apply_along_axis(lambda m: np.convolve(m,filter_weights, mode="valid")[1:-1], axis=1, arr=n)
df_usefulness["total"] = df_usefulness[STAGES].sum(axis=1)

In [20]:
df.sort_values(by="total", ascending=False)

,375,750,1125,1500,1875,2250,total,text
16533,14.392889,15.303111,13.880889,15.246222,15.018666,15.189333,89.031113,"A:\tYeah,"
12500,13.880889,15.388445,15.331555,14.933333,14.279111,14.705778,88.519112,A:\tYeah.
17424,15.616000,13.966222,14.136889,14.620444,15.616000,14.392889,88.348442,B:\tIt was about four or five years ago.
17741,14.677333,15.018666,14.307555,14.734222,15.132444,14.279111,88.149338,"B:\tBut you know, for, for what."
4125,13.738667,15.701333,13.880889,14.734222,15.502222,14.592000,88.149338,"B:\tthrow the lids and everything out,"
...,...,...,...,...,...,...,...,...
11931,-0.581333,-0.452444,-0.672000,-0.577778,-0.504000,-0.540444,-3.328000,A:\tFood
632,-0.632889,-0.531556,-0.659556,-0.535111,-0.554667,-0.506667,-3.420444,A:\tI like it a lot.
9611,-0.461333,-0.725333,-0.547556,-0.486222,-0.572444,-0.712889,-3.505778,B:\tGetting ready to eat chicken too.
1223,-0.698667,-0.780444,-0.755556,-0.604444,-0.471111,-0.705778,-4.016000,"B:\tAlzheimer,"


In [21]:
df_usefulness = df_usefulness.sort_values(by="total", ascending=False)
df_usefulness

,375,750,1125,1500,1875,2250,total,text
16533,12.709056,15.162583,9.928205,14.253641,14.867397,14.755794,81.676676,"A:\tYeah,"
17424,13.628410,14.262896,10.111307,14.034410,15.153281,14.158458,81.348762,B:\tIt was about four or five years ago.
12500,12.595202,15.060796,10.965784,14.029112,14.240495,14.362292,81.253681,A:\tYeah.
2812,13.008173,14.726002,10.070618,13.658714,14.950045,14.559200,80.972752,"A:\tYeah,"
17741,12.986723,14.916711,10.233375,13.962895,14.772185,14.044183,80.916073,"B:\tBut you know, for, for what."
...,...,...,...,...,...,...,...,...
11931,-0.532692,-0.471417,-0.480643,-0.555294,-0.513515,-0.540037,-3.093597,A:\tFood
632,-0.567402,-0.534217,-0.471742,-0.532987,-0.542469,-0.511269,-3.160087,A:\tI like it a lot.
9611,-0.425215,-0.684208,-0.391635,-0.461358,-0.582154,-0.636887,-3.181457,B:\tGetting ready to eat chicken too.
1223,-0.631149,-0.713144,-0.540406,-0.599865,-0.524518,-0.672260,-3.681342,"B:\tAlzheimer,"


In [22]:
df_usefulness.sort_values(by=STAGES[0], ascending=False)[0:NUM_DOCS_STAGE]

,375,750,1125,1500,1875,2250,total,text
1715,12.617180,13.446339,11.474402,13.315261,14.753205,14.194111,79.800498,"B:\tNow, meanwhile, I got, had a, a building b..."
12933,12.161911,13.704438,11.108196,13.561226,14.398756,14.157224,79.091752,B:\tIt takes time
7025,13.199489,13.633243,11.108196,13.671164,14.046282,14.495741,80.154116,"B:\tYep,"
1211,12.157515,13.563226,11.006474,13.069242,13.549479,13.731311,77.077248,B:\tUh.
17035,11.822104,14.138294,10.965784,12.132133,13.501498,13.229812,75.789625,"B:\tNo,"
...,...,...,...,...,...,...,...,...
5021,5.235859,5.990112,4.547036,5.918230,5.991682,5.728502,33.411422,"B:\tNo,"
17133,5.263902,5.866524,4.536864,5.833235,6.274031,6.186561,33.961118,"A:\tBut,"
13535,5.060455,6.016824,4.536864,5.743066,5.932120,5.844004,33.133333,"B:\tWhat, what are you looking at?"
10005,5.721657,6.165428,4.536864,5.873360,6.288031,5.719939,34.305279,B:\tI'm sorry.


In [23]:

df_usefulness_ = df_usefulness.copy()
for stage in STAGES:
    pd.concat([df_usefulness_,df_usefulness.sort_values(by=STAGES[1], ascending=False)]).drop_duplicates(keep="first")[0:2*NUM_DOCS_STAGE]

In [24]:
df_usefulness

,375,750,1125,1500,1875,2250,total,text
16533,12.709056,15.162583,9.928205,14.253641,14.867397,14.755794,81.676676,"A:\tYeah,"
17424,13.628410,14.262896,10.111307,14.034410,15.153281,14.158458,81.348762,B:\tIt was about four or five years ago.
12500,12.595202,15.060796,10.965784,14.029112,14.240495,14.362292,81.253681,A:\tYeah.
2812,13.008173,14.726002,10.070618,13.658714,14.950045,14.559200,80.972752,"A:\tYeah,"
17741,12.986723,14.916711,10.233375,13.962895,14.772185,14.044183,80.916073,"B:\tBut you know, for, for what."
...,...,...,...,...,...,...,...,...
11931,-0.532692,-0.471417,-0.480643,-0.555294,-0.513515,-0.540037,-3.093597,A:\tFood
632,-0.567402,-0.534217,-0.471742,-0.532987,-0.542469,-0.511269,-3.160087,A:\tI like it a lot.
9611,-0.425215,-0.684208,-0.391635,-0.461358,-0.582154,-0.636887,-3.181457,B:\tGetting ready to eat chicken too.
1223,-0.631149,-0.713144,-0.540406,-0.599865,-0.524518,-0.672260,-3.681342,"B:\tAlzheimer,"


### With convolution

In [29]:
df_usefulness_ = df_usefulness.copy()
dfs_selection = []
for stage in STAGES:
    dfs_selection.append(df_usefulness_.nlargest(NUM_DOCS_STAGE, columns=stage))
    df_usefulness_ = df_usefulness_.nsmallest(len(df_usefulness_) - NUM_DOCS_STAGE, columns=stage)
 
print(len(df_usefulness_))
for stage_id, d in enumerate(dfs_selection):
    print(os.path.join(args["curriculum_output_folder"],"{}.train".format(stage_id)))
    with open(os.path.join(args["curriculum_output_folder"],"{}.train".format(stage_id)), "w") as f:
        for line in d["text"]:
            f.write(line + "\n") 
    display(d)
    

0
./10MCurriculum/0.train


,375,750,1125,1500,1875,2250,total,text
1715,12.617180,13.446339,11.474402,13.315261,14.753205,14.194111,79.800498,"B:\tNow, meanwhile, I got, had a, a building b..."
7025,13.199489,13.633243,11.108196,13.671164,14.046282,14.495741,80.154116,"B:\tYep,"
12933,12.161911,13.704438,11.108196,13.561226,14.398756,14.157224,79.091752,B:\tIt takes time
1211,12.157515,13.563226,11.006474,13.069242,13.549479,13.731311,77.077248,B:\tUh.
12500,12.595202,15.060796,10.965784,14.029112,14.240495,14.362292,81.253681,A:\tYeah.
...,...,...,...,...,...,...,...,...
16498,5.530857,6.036734,4.547036,5.491532,5.573217,5.673077,32.852453,B:\tthat's
6070,5.996311,6.986042,4.536864,6.336880,6.757421,6.460320,37.073837,A:\tI went to Mexico City one time and stayed.
6645,5.762347,6.962848,4.536864,6.259633,6.584095,6.476144,36.581931,"A:\tYeah,"
16997,5.711485,6.469782,4.536864,6.287952,6.825439,6.379987,36.211509,A:\tokay.


./10MCurriculum/1.train


,375,750,1125,1500,1875,2250,total,text
11545,6.332550,6.981087,4.455486,6.996367,6.953864,7.269528,38.988881,A:\tSo are you a professional musician?
3994,6.289111,6.485538,4.486002,6.284703,6.576665,6.088545,36.210564,"A:\tBut, uh, I find myself listening to popula..."
11248,6.230827,6.114432,4.455486,6.218873,6.307631,6.497967,35.825216,A:\tI had that almost similar situation.
2804,6.151646,6.694353,4.506347,6.526451,6.996293,6.998442,37.873532,"A:\tyou know, I mean, there's even the C N N c..."
1997,6.144500,6.484020,4.435141,6.288900,6.728596,6.593940,36.675097,"A:\tand, and, um, its concentration,"
...,...,...,...,...,...,...,...,...
13886,2.393736,2.595760,1.958176,2.647175,2.589395,2.642217,14.826459,"B:\tOh, I heard that was excellent."
9990,2.389340,2.605092,1.856452,2.618032,2.557559,2.537405,14.563882,A:\tand when they go on vacation a lot of time...
11642,2.388932,2.372768,1.729298,2.324718,2.323871,2.363118,13.502706,"B:\tWow,"
14518,2.388097,2.652848,2.039554,2.530935,2.670695,2.548458,14.830589,"B:\tSo, that,"


./10MCurriculum/2.train


...

,375,750,1125,1500,1875,2250,total,text
1452,0.314939,0.250933,0.258441,0.279923,0.372184,0.474183,1.950603,B:\tbut you'll stay there for twenty years.
14153,0.293924,0.429287,0.275289,0.348204,0.498060,0.472603,2.317367,B:\tIf you don't have a frost you'll be all right
7550,0.420079,0.479512,0.352217,0.356089,0.556230,0.458825,2.622953,A:\tIt wasn't.
3240,0.272528,0.476852,0.359847,0.339475,0.426986,0.447209,2.322897,A:\tRight.
12925,0.376420,0.331058,0.251130,0.302792,0.202387,0.443126,1.906912,"B:\tIt was, they're very good, very good."
...,...,...,...,...,...,...,...,...
4675,0.039142,0.050852,0.028928,0.034510,0.034964,0.032455,0.220851,A:\tand so I'm pushing that on my daughter bec...
5257,0.036807,0.029090,0.024557,0.033392,0.031392,0.032438,0.187676,"B:\tAt the moment I've got, uh, Cadillac Cimar..."
10311,0.005570,0.029065,0.012199,0.025910,0.025428,0.032431,0.130602,A:\tThey weren't stealing it out of mine so mu...
1006,0.064761,0.032548,0.017285,0.080571,0.064877,0.032430,0.292473,"B:\tand finally they realized that they were, ..."


./10MCurriculum/4.train


,375,750,1125,1500,1875,2250,total,text
12071,0.150436,0.130211,0.164347,0.126686,0.146895,0.024688,0.743264,"B:\tAnd they don't learn the communications,"
3763,0.056062,0.013709,-0.075339,0.004366,0.145895,-0.047077,0.097617,B:\tI have three children
2713,0.053474,0.040108,-0.010172,0.070755,0.136194,0.008090,0.298448,A:\tthe library has a paper.
6322,0.115408,0.078568,0.038623,0.074847,0.135703,-0.048682,0.394468,"A:\tWhat do you do with it,"
8216,0.162667,0.094985,0.130333,0.092856,0.131843,0.016229,0.628914,A:\tyou know what I mean?
...,...,...,...,...,...,...,...,...
4148,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"B:\tOne,"
10237,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,B:\tUh-huh.
10252,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,B:\tyeah.
10251,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"B:\tyeah,"


./10MCurriculum/5.train


,375,750,1125,1500,1875,2250,total,text
13843,0.034101,0.161636,0.104266,0.066394,-0.037062,-0.030010,0.299325,"A:\tI know that it's scary,"
12163,0.005171,0.152065,-0.041722,0.064705,-0.014260,-0.017429,0.148529,"B:\tWell, vicarious learning is a wonderful th..."
13256,-0.044206,0.142094,-0.052690,-0.043432,-0.068321,0.007454,-0.059101,"A:\tand they'll say,"
16730,-0.035822,0.112242,0.021974,0.018561,-0.000394,0.026562,0.143123,A:\tan I don't care for rap music either.
243,0.076045,0.087370,0.071524,0.122553,-0.032767,0.031830,0.356556,B:\tno memory.
...,...,...,...,...,...,...,...,...
17459,-0.476028,-0.503402,-0.305170,-0.429509,-0.384017,-0.486586,-2.584712,A:\tI guess that could happen.
632,-0.567402,-0.534217,-0.471742,-0.532987,-0.542469,-0.511269,-3.160087,A:\tI like it a lot.
9611,-0.425215,-0.684208,-0.391635,-0.461358,-0.582154,-0.636887,-3.181457,B:\tGetting ready to eat chicken too.
1223,-0.631149,-0.713144,-0.540406,-0.599865,-0.524518,-0.672260,-3.681342,"B:\tAlzheimer,"


In [26]:
from datasets import load_dataset
dataset_c = load_dataset("text", data_dir=args["curriculum_output_folder"])

Generating train split: 18000 examples [00:00, 647969.10 examples/s]


In [34]:
# assert all([t in dataset["train"]["text"] for t in dataset_c["train"]["text"]])

In [52]:
dataset_c["train"][0]["text"] in dataset["train"]["text"]

True

### No convolution

In [27]:
df_usefulness_ = df.copy()
dfs_selection = []
for stage in STAGES:
    dfs_selection.append(df_usefulness_.nlargest(NUM_DOCS_STAGE, columns=stage))
    df_usefulness_ = df_usefulness_.nsmallest(len(df_usefulness_) - NUM_DOCS_STAGE, columns=stage)
    print(len(df_usefulness_))
print(len(df_usefulness_))
for a in dfs_selection:
    display(a)

15000
12000
9000
6000
3000
0
0


,375,750,1125,1500,1875,2250,total,text
1715,13.738667,12.970667,16.042667,13.909333,15.075556,14.705778,86.442665,"B:\tNow, meanwhile, I got, had a, a building b..."
7025,14.677333,13.454223,15.530666,14.222222,13.966222,14.990222,86.840889,"B:\tYep,"
12933,13.226666,13.454223,15.530666,14.421333,14.592000,14.592000,85.816895,B:\tIt takes time
1211,13.255111,13.568000,15.388445,13.738667,13.568000,14.165334,83.683556,B:\tUh.
12500,13.880889,15.388445,15.331555,14.933333,14.279111,14.705778,88.519112,A:\tYeah.
...,...,...,...,...,...,...,...,...
16498,6.186666,6.215111,6.357333,5.632000,5.560889,5.802667,35.754665,B:\tthat's
580,6.357333,6.243556,6.343111,6.200889,6.314667,6.471111,37.930668,B:\tyeah
854,6.072889,6.784000,6.343111,6.272000,6.684444,5.902222,38.058666,"A:\tSo,"
3143,6.456889,6.215111,6.343111,6.656000,6.997334,6.257778,38.926220,A:\tNo income tax and no sales tax.


,375,750,1125,1500,1875,2250,total,text
11545,7.338666,6.983111,6.229333,7.466667,6.883555,7.495111,42.396442,A:\tSo are you a professional musician?
3994,7.267556,6.471111,6.272000,6.485333,6.784000,6.087111,39.367111,"A:\tBut, uh, I find myself listening to popula..."
11248,7.196445,6.030222,6.229333,6.414222,6.272000,6.684444,38.826668,A:\tI had that almost similar situation.
1997,7.082667,6.400000,6.200889,6.542222,6.826667,6.798222,39.850666,"A:\tand, and, um, its concentration,"
2804,7.068444,6.584889,6.300445,6.869333,7.054222,7.281778,41.159111,"A:\tyou know, I mean, there's even the C N N c..."
...,...,...,...,...,...,...,...,...
7769,2.688000,2.880000,2.908444,2.801778,2.887111,2.965333,17.130667,"B:\tuh. Gee, I'll have to try to catch that so..."
9132,2.680889,2.552889,2.567111,2.375111,2.851556,2.887111,15.914667,B:\tit's definitely unfortunate
4812,2.680889,2.531556,2.680889,2.560000,2.659556,2.488889,15.601777,A:\tI can feel just the top of my scalp gettin...
13886,2.680889,2.595556,2.737778,2.816000,2.581333,2.680889,16.092443,"B:\tOh, I heard that was excellent."


,375,750,1125,1500,1875,2250,total,text
12919,2.645333,2.787555,3.057778,3.299556,2.922667,3.285333,17.998222,B:\tAnd I thaw it
1762,2.624000,2.880000,2.972445,3.157333,2.894222,2.673778,17.201778,"A:\tI mean, uh, I say it might go for another ..."
5182,2.666667,2.780444,2.894222,3.057778,2.723556,2.645333,16.768000,B:\tI think that could be true too.
12501,2.602667,2.574222,2.780444,2.993778,2.929778,2.993778,16.874666,"B:\tand, uh, they are real easy to cook"
2303,2.631111,2.759111,2.574222,2.993778,2.545778,2.709333,16.213335,"A:\tOh, well, that's nice."
...,...,...,...,...,...,...,...,...
2111,0.168000,0.310222,0.304000,0.378667,0.356444,0.300444,1.817778,B:\tThey're still doing business. Big business...
14519,0.250222,0.349333,0.342222,0.378667,0.375111,0.257778,1.953333,"B:\tyou mean,"
4133,0.311111,0.336000,0.322667,0.378667,0.346667,0.363556,2.058667,"B:\tUm, and then, of course, uh, cans, coke ca..."
2899,0.400889,0.406222,0.392889,0.378667,0.388444,0.452444,2.419556,"A:\tand they're, you know, already out with nu..."


,375,750,1125,1500,1875,2250,total,text
1452,0.352444,0.200444,0.361333,0.274889,0.343111,0.552889,2.085111,B:\tbut you'll stay there for twenty years.
463,0.514667,0.378667,0.432889,0.352000,0.382222,0.539556,2.600000,A:\tRight.
14153,0.317333,0.406222,0.384889,0.376889,0.517333,0.528000,2.530667,B:\tIf you don't have a frost you'll be all right
3609,0.323556,0.335111,0.328000,0.093222,0.400000,0.526222,2.006111,B:\tYou're just like me.
8873,0.282667,0.340000,0.275111,0.274667,0.342222,0.517333,2.032000,B:\tbut I guess I do
...,...,...,...,...,...,...,...,...
6515,0.043000,0.032444,0.013889,0.030111,0.023389,0.033444,0.176278,"A:\tbut just like everything else, I was real ..."
4735,0.029056,0.036333,0.036222,0.033000,0.033111,0.033444,0.201167,"A:\tthere's a there's a place, um, half a mile..."
8260,0.032833,0.032611,0.032278,0.036278,0.033000,0.033444,0.200444,"B:\tI just, uh, I'd like to believe that there..."
13844,0.032500,0.039000,0.028500,0.039778,0.036778,0.033444,0.210000,A:\tand yet I hate the government constantly t...


,375,750,1125,1500,1875,2250,total,text
2881,0.155778,-0.022528,0.024910,0.226222,0.353778,0.022611,0.760771,"B:\tWhat do you,"
3763,0.104000,-0.028889,-0.105333,-0.010222,0.219556,-0.068333,0.110778,B:\tI have three children
7620,0.139556,0.023889,0.198667,0.042222,0.218222,0.027111,0.649667,B:\tWest
14972,0.087556,0.161333,0.193333,0.176889,0.204444,-0.000667,0.822889,"B:\tOh, it was excellent,"
6322,0.148222,0.063611,0.054000,0.064000,0.201778,-0.098222,0.433389,"A:\tWhat do you do with it,"
...,...,...,...,...,...,...,...,...
9265,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"B:\tUh, but I agree,"
9267,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,A:\tUh-huh
9281,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,B:\tor he's innocent.
9283,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,A:\tRight.


,375,750,1125,1500,1875,2250,total,text
13843,0.012222,0.238222,0.145778,0.077444,-0.045139,-0.067167,0.361361,"A:\tI know that it's scary,"
13256,-0.043889,0.222667,-0.073667,-0.043778,-0.095556,0.027500,-0.006722,"A:\tand they'll say,"
12163,0.021417,0.217333,-0.058333,0.090222,-0.016167,-0.046000,0.208472,"B:\tWell, vicarious learning is a wonderful th..."
16730,-0.057556,0.156000,0.030722,0.037333,-0.009917,0.031833,0.188417,A:\tan I don't care for rap music either.
8180,-0.026611,0.152889,-0.097111,0.041111,-0.118222,-0.047528,-0.095472,A:\twhere does she live?
...,...,...,...,...,...,...,...,...
17530,-0.396444,-0.540444,-0.276889,-0.367111,-0.311111,-0.385778,-2.277778,B:\tI guess so.
17459,-0.561778,-0.545778,-0.426667,-0.427556,-0.347556,-0.512889,-2.822222,A:\tI guess that could happen.
9611,-0.461333,-0.725333,-0.547556,-0.486222,-0.572444,-0.712889,-3.505778,B:\tGetting ready to eat chicken too.
7579,-0.954667,-0.737778,-0.695111,-0.632889,-0.915556,-0.652444,-4.588444,A:\tProbably needed some cold beer with it


In [28]:
df_usefulness

,375,750,1125,1500,1875,2250,total,text
16533,12.709056,15.162583,9.928205,14.253641,14.867397,14.755794,81.676676,"A:\tYeah,"
17424,13.628410,14.262896,10.111307,14.034410,15.153281,14.158458,81.348762,B:\tIt was about four or five years ago.
12500,12.595202,15.060796,10.965784,14.029112,14.240495,14.362292,81.253681,A:\tYeah.
2812,13.008173,14.726002,10.070618,13.658714,14.950045,14.559200,80.972752,"A:\tYeah,"
17741,12.986723,14.916711,10.233375,13.962895,14.772185,14.044183,80.916073,"B:\tBut you know, for, for what."
...,...,...,...,...,...,...,...,...
11931,-0.532692,-0.471417,-0.480643,-0.555294,-0.513515,-0.540037,-3.093597,A:\tFood
632,-0.567402,-0.534217,-0.471742,-0.532987,-0.542469,-0.511269,-3.160087,A:\tI like it a lot.
9611,-0.425215,-0.684208,-0.391635,-0.461358,-0.582154,-0.636887,-3.181457,B:\tGetting ready to eat chicken too.
1223,-0.631149,-0.713144,-0.540406,-0.599865,-0.524518,-0.672260,-3.681342,"B:\tAlzheimer,"


In [29]:
def curriculize(df):
    

SyntaxError: incomplete input (1706643163.py, line 2)

In [415]:
np.convolve(n[1,:],filter_weights, mode="full").shape

(9,)

In [394]:
c = np.convolve(n[0,:],filter_weights, mode="full")
c

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -4.35944797e-04,
       -1.58928861e-04, -4.67267044e-05, -1.96619876e-05, -1.65867393e-06,
        1.18949971e-07])

In [335]:

a = np.arange(10)
a = np.vstack((a,a)).T
print(a)
filt = np.ones(3)

np.apply_along_axis(lambda m: np.convolve(m, filt, mode='full'), axis=0, arr=a)

[[0 0]
 [1 1]
 [2 2]
 [3 3]
 [4 4]
 [5 5]
 [6 6]
 [7 7]
 [8 8]
 [9 9]]


array([[ 0.,  0.],
       [ 1.,  1.],
       [ 3.,  3.],
       [ 6.,  6.],
       [ 9.,  9.],
       [12., 12.],
       [15., 15.],
       [18., 18.],
       [21., 21.],
       [24., 24.],
       [17., 17.],
       [ 9.,  9.]])

In [336]:
np.array([0,1,0])

array([0, 1, 0])

In [337]:
df.sort_values(by="50000", ascending=False)

,100000,50000,150000,total,text
document_id,,,,,
744,0.000000,0.008670,0.000000,0.008670,"""No."""
9724,0.000350,0.008310,0.000344,0.009005,"""Had to! I’d have seen them shot first!"" Tom ..."
2829,0.000264,0.008285,0.000000,0.008549,"""No."""
973,0.001230,0.008178,0.000000,0.009408,"""What is it?"" cried Pen."
426,-0.000200,0.008147,0.000031,0.007978,"""Think so?"""
...,...,...,...,...,...
2046,0.001661,-0.000650,0.000257,0.001268,CHAPTER TWENTY NINE.
5792,0.000000,-0.000829,0.000172,-0.000657,The boys looked up and laughed at the irate li...
6904,0.000374,-0.001018,0.000035,-0.000610,The next minute two pairs of childish arms wer...


In [41]:
# slow = get_mean_influence_at_checkpoint("/data/loriss21dm/babylm/results/100000_65000_66000")
# fast = get_mean_influence_at_checkpoint_einsum("/data/loriss21dm/babylm/results/100000_65000_66000")
# cos = torch.nn.CosineSimilarity(dim=-1, eps=1e-6)
# cos(fast,slow)

SyntaxError: incomplete input (2249411127.py, line 1)

In [16]:
# with torch.no_grad():
#     result = get_mean_influence_at_checkpoint_at("/data/loriss21dm/babylm/results/100000_65000_66000").shape
#     print(result.shape)

In [17]:
77590 / 95830

0.80966294479808

In [4]:
result = None

In [22]:
len(dataset["train"])

676014

In [49]:
gradients_at_checkpoint.shape

torch.Size([676014])

In [11]:
from itertools import product

In [80]:
len(dataset["train"],)*len(dataset["train"],)

456994928196

In [84]:
len((get_all_chunks("/data/loriss21dm/babylm/results/100000_65000_66000")))

677

In [11]:
gradients_at_checkpoint = torch.zeros((len(dataset["train"],)))
for chunk_path_a in get_all_chunks("/data/loriss21dm/babylm/results/100000_65000_66000"):
    _, start_id_a, stop_id_a = chunk_path_a.split( "_")
    start_id_a = int(start_id_a)
    stop_id_a = int(stop_id_a)
    chunk_a = torch.load(chunk_path_a, weights_only=True,map_location="cpu").flatten(1)
    for chunk_path_b in get_all_chunks("/data/loriss21dm/babylm/results/100000_65000_66000"):
        _, start_id_b, stop_id_b = chunk_path_b.split( "_")
    
        start_id_b = int(start_id_b)
        stop_id_b = int(stop_id_b)

        print(chunk_path_a, chunk_path_b)
        
        chunk_b = torch.load(chunk_path_b, weights_only=True,map_location="cpu").flatten(1)
    # print(start_id_a, stop_id_a, start_id_b, stop_id_b)
        # print((torch.einsum('ik, kj -> i', chunk_a, chunk_b.T) / gradients_at_checkpoint.shape[0]).shape)
        # print(gradients_at_checkpoint[start_id_a:stop_id_a].shape)
        # print(torch.einsum('ik, kj -> i', chunk_a, chunk_b.T).shape)
        gradients_at_checkpoint[start_id_a:stop_id_a] += torch.einsum('ik, kj -> i', chunk_a, chunk_b.T) 
gradients_at_checkpoint

/data/loriss21dm/babylm/results/100000_0_1000 /data/loriss21dm/babylm/results/100000_0_1000
/data/loriss21dm/babylm/results/100000_0_1000 /data/loriss21dm/babylm/results/100000_1000_2000
/data/loriss21dm/babylm/results/100000_0_1000 /data/loriss21dm/babylm/results/100000_2000_3000
/data/loriss21dm/babylm/results/100000_1000_2000 /data/loriss21dm/babylm/results/100000_0_1000
/data/loriss21dm/babylm/results/100000_1000_2000 /data/loriss21dm/babylm/results/100000_1000_2000
/data/loriss21dm/babylm/results/100000_1000_2000 /data/loriss21dm/babylm/results/100000_2000_3000
/data/loriss21dm/babylm/results/100000_2000_3000 /data/loriss21dm/babylm/results/100000_0_1000
/data/loriss21dm/babylm/results/100000_2000_3000 /data/loriss21dm/babylm/results/100000_1000_2000
/data/loriss21dm/babylm/results/100000_2000_3000 /data/loriss21dm/babylm/results/100000_2000_3000


tensor([-108.3750,  264.9062,  436.5000,  ...,   22.7500,   28.6832,
          32.2500])

In [49]:
torch.allclose((gradients_at_checkpoint / 3000).float(), slow.flatten().float(), rtol=0.1)
cos = torch.nn.CosineSimilarity(dim=-1, eps=1e-6)
cos((gradients_at_checkpoint / 3000).float(),slow.flatten().float())

tensor(1.0000)

In [46]:
(gradients_at_checkpoint / 3000).float()[2000:]

tensor([ 1.4850e-01,  8.4601e-03,  9.7833e-02,  4.5167e-02,  0.0000e+00,
         1.2292e-02,  2.4633e-01,  5.3978e-03,  0.0000e+00,  9.6896e-02,
         3.3250e-02,  9.8625e-02,  7.7773e-03,  3.7458e-02,  1.4412e-01,
         7.8302e-02,  3.8052e-02,  3.8573e-02,  7.4396e-02,  1.7613e-01,
         2.1374e-01,  1.0042e-02,  3.5327e-03,  2.4042e-02,  2.4813e-02,
         0.0000e+00,  5.5245e-02,  1.3771e-02,  1.1760e-02, -4.0417e-03,
         3.2250e-02,  3.5969e-01,  3.0286e-02,  1.8848e-01,  1.6517e-01,
         0.0000e+00,  3.1042e-02,  0.0000e+00,  1.6567e-01,  9.4896e-02,
         2.4958e-02,  1.8042e-02,  2.3238e-02,  1.0372e-02,  1.7880e-02,
         2.7904e-02,  1.5000e-01,  0.0000e+00,  1.1625e-02,  3.9130e-02,
         1.2589e-02,  2.7198e-02,  2.5135e-02,  1.6161e-02,  2.7250e-02,
         2.6323e-02,  2.1198e-02,  2.6313e-01,  1.1548e-01,  2.1115e-02,
         3.6875e-03,  3.2563e-02,  1.9875e-02,  1.2917e-02,  3.0167e-02,
         2.7552e-03,  4.4417e-02,  2.6875e-02,  1.7

In [45]:
slow[0,2000:]

tensor([ 1.4844e-01,  8.4229e-03,  9.8145e-02,  4.5410e-02,  0.0000e+00,
         1.2268e-02,  2.4609e-01,  5.4932e-03,  0.0000e+00,  9.7168e-02,
         3.3203e-02,  9.8633e-02,  7.7820e-03,  3.7598e-02,  1.4453e-01,
         7.8613e-02,  3.8086e-02,  3.8574e-02,  7.4219e-02,  1.7578e-01,
         2.1387e-01,  1.0010e-02,  3.5400e-03,  2.4048e-02,  2.5024e-02,
         0.0000e+00,  5.4932e-02,  1.3794e-02,  1.1780e-02, -4.0588e-03,
         3.2227e-02,  3.5938e-01,  3.0396e-02,  1.8848e-01,  1.6504e-01,
         0.0000e+00,  3.1128e-02,  0.0000e+00,  1.6504e-01,  9.5215e-02,
         2.4902e-02,  1.7944e-02,  2.3193e-02,  1.0376e-02,  1.7822e-02,
         2.7832e-02,  1.4941e-01,  0.0000e+00,  1.1597e-02,  3.9062e-02,
         1.2512e-02,  2.7100e-02,  2.5146e-02,  1.6235e-02,  2.7344e-02,
         2.6367e-02,  2.1240e-02,  2.6367e-01,  1.1572e-01,  2.1118e-02,
         3.6774e-03,  3.2715e-02,  1.9897e-02,  1.2878e-02,  3.0273e-02,
         2.7618e-03,  4.4434e-02,  2.6855e-02,  1.7

In [32]:
(gradients_at_checkpoint / 3000).float()

tensor([-0.0361,  0.0883,  0.1455,  ...,  0.0076,  0.0096,  0.0108])

In [31]:
(gradients_at_checkpoint / 3000).float() == slow.float()

tensor([[False, False, False,  ..., False, False, False]])

In [33]:
slow.float()

tensor([[-0.0361,  0.0884,  0.1455,  ...,  0.0076,  0.0096,  0.0107]])

In [21]:
slow

tensor([[-0.0361,  0.0884,  0.1455,  ...,  0.0076,  0.0096,  0.0107]],
       dtype=torch.bfloat16)

In [78]:
torch.all(gradients_at_checkpoint == 677)

tensor(True)

In [81]:
gradients_at_checkpoint.shape

torch.Size([676014])

In [ ]:
#with torch.no_grad(): # TODO redundant
for chunk in tqdm(get_all_chunks("/data/loriss21dm/babylm/results/100000_65000_66000"), desc="Loading checkpoint chunks from disk"):
    _, start_id, stop_id = chunk.split( "_")
    
# gradients_at_checkpoint.shape

In [ ]:
4tfeu0d9üp

In [ ]:
#with torch.no_grad(): # TODO redundant
for chunk in tqdm(get_all_chunks("/data/loriss21dm/babylm/results/100000_65000_66000"), desc="Loading checkpoint chunks from disk"):
    _, start_id, stop_id = chunk.split( "_")
gradients_at_checkpoint = 
[torch.load(chunk, weights_only=True,map_location="cpu") ]
# gradients_at_checkpoint.shape

Loading checkpoint chunks from disk: 100%|████████████████████████████████████████| 677/677 [05:19<00:00,  2.12it/s]


: 

: 

: 

: 

In [23]:
aaaaaw
def get_mean_influence_at_checkpoint_einsum_batched(checkpoint_path, batch_size=100000):

    # gradients_at_checkpoint = torch.concat([torch.load(chunk, weights_only=True,map_location="cpu") for chunk in tqdm(get_all_chunks(checkpoint_path), desc="Loading checkpoint chunks from disk")]).flatten(1)

    num_batches = (gradients_at_checkpoint.shape[0] + batch_size - 1) // batch_size  # Calculate number of batches
    results = []


    for i in range(num_batches):

        start = i * batch_size
        end = min((i + 1) * batch_size, gradients_at_checkpoint.shape[0])
        gradients_batch = gradients_at_checkpoint[start:end]
        print("gradients_batch",gradients_batch.shape)
        print("gradients_at_checkpoint.T", gradients_at_checkpoint.T.shape)
      
        batch_result = torch.einsum('ik, kj -> i', gradients_batch, gradients_at_checkpoint.T)
        results.append(batch_result)  
        print("batch_result", batch_result.shape)

    return (torch.concat(results) / gradients_at_checkpoint.shape[0]).unsqueeze(0)


In [24]:
slower = get_mean_influence_at_checkpoint_einsum_batched("/data/loriss21dm/babylm/results/100000_65000_66000")
slower.shape

gradients_batch torch.Size([200000, 393216])
gradients_at_checkpoint.T torch.Size([393216, 200000])
batch_result torch.Size([200000])


torch.Size([1, 200000])

In [25]:
# fast = (torch.einsum('ik, kj -> i', gradients_at_checkpoint, gradients_at_checkpoint.T) / gradients_at_checkpoint.shape[0]).unsqueeze(0)

In [26]:
# fast[0][100:200]

tensor([ 1.0449e-01, -7.6294e-03,  0.0000e+00,  1.6357e-02,  3.6621e-02,
         1.0742e-02, -3.3264e-03,  2.0898e-01,  4.6387e-02, -4.6997e-03,
         7.6904e-03,  3.0060e-03,  4.1199e-03,  1.5381e-02, -1.1978e-03,
        -1.3062e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  8.5449e-03,
         1.4465e-02,  3.4668e-02,  1.2970e-03,  6.6528e-03,  0.0000e+00,
         3.0212e-03,  9.7046e-03,  6.3419e-05,  9.2285e-02,  2.5787e-03,
         4.3457e-02,  9.9121e-02,  1.4725e-03,  1.2402e-01,  6.4392e-03,
         1.5527e-01,  0.0000e+00,  7.0190e-04,  0.0000e+00,  1.8311e-03,
        -2.6367e-02,  3.5248e-03, -5.6763e-03,  5.0049e-03,  2.7832e-02,
         1.5747e-02,  3.4668e-02,  3.1128e-03,  3.8477e-01,  9.2285e-02,
         8.3618e-03,  1.0132e-02,  1.9775e-02,  3.2349e-03,  4.7607e-02,
         5.6458e-03,  1.5869e-02,  1.8387e-03,  0.0000e+00,  2.4292e-02,
         2.1289e-01,  8.9844e-02,  1.2695e-02,  1.4465e-02,  1.8066e-01,
         5.3711e-03,  1.6357e-02,  3.4668e-02,  7.8

In [26]:
torch.allclose(slower, fast, rtol=0.1)

True